In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', None)

plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

In [110]:
data_string=r"C:\Users\am513\PS-88-21-DEV\lab\aaronfolder\DSUS\Data\NHIS2009_clean.dta" 
df=pd.read_stata(data_string)
df
#df.to_csv("NHIS2009_clean.csv")

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,...,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1
0,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,1,...,14.0,1.0,4.0,19282.931641,17500.0,1.0,0.0,0.0,0.0,NaN
1,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,14.0,0.0,4.0,19282.931641,17500.0,1.0,1.0,0.0,1.0,NaN
2,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,4.0,0.0,5.0,19282.931641,17500.0,0.0,0.0,0.0,1.0,NaN
3,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,NaN,0.0,5.0,19282.931641,17500.0,0.0,0.0,0.0,1.0,NaN
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,11.0,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80629,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,...,14.0,1.0,2.0,167844.531250,150000.0,1.0,0.0,0.0,2.0,NaN
80630,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,...,14.0,0.0,5.0,167844.531250,150000.0,1.0,0.0,0.0,2.0,NaN
80631,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,...,12.0,1.0,4.0,167844.531250,150000.0,1.0,0.0,0.0,2.0,NaN
80632,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41176,...,11.0,0.0,5.0,167844.531250,150000.0,0.0,0.0,0.0,2.0,NaN


In [111]:
df.describe()

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,...,yedu,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1
count,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,...,74639.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,80634.000000,15093.000000
mean,2009.472290,19292.210938,41708.820312,61106.566406,86013.531250,167919.546875,25375.941406,109663.078125,70811.601562,20422.576345,...,11.350019,0.442084,3.884143,68465.328125,64391.335938,0.586142,0.374482,0.241920,0.818389,0.854568
std,0.472091,9.275610,29.468706,3.595120,27.765621,75.035095,22.388733,62.218945,23.039238,11907.151576,...,4.636348,0.496557,1.052505,54488.144531,48474.695312,0.492567,0.484127,0.428243,0.863409,0.352545
min,2009.000000,19282.931641,41679.343750,61102.972656,85985.781250,167844.531250,25398.326172,109725.273438,70834.640625,1.000000,...,0.000000,0.000000,1.000000,19282.931641,17500.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,19282.931641,41679.343750,61102.972656,85985.781250,167844.531250,25398.326172,109725.273438,70834.640625,10047.000000,...,9.000000,0.000000,3.000000,19282.931641,17500.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2009.000000,19282.931641,41679.343750,61102.972656,85985.781250,167844.531250,25398.326172,109725.273438,70834.640625,20350.000000,...,12.000000,0.000000,4.000000,61102.972656,62500.000000,1.000000,0.000000,0.000000,1.000000,1.000000
75%,2009.000000,19282.931641,41679.343750,61102.972656,85985.781250,167844.531250,25398.326172,109725.273438,70834.640625,30757.000000,...,14.000000,1.000000,5.000000,85985.781250,87500.000000,1.000000,1.000000,0.000000,2.000000,1.000000
max,2009.000000,19282.931641,41679.343750,61102.972656,85985.781250,167844.531250,25398.326172,109725.273438,70834.640625,41177.000000,...,18.000000,1.000000,5.000000,167844.531250,150000.000000,1.000000,1.000000,1.000000,2.000000,1.000000


In [162]:
df.astype({'marradult': 'float64'}).dtypes;

In [113]:
df2=df.query("marradult==1 & perweight!=0") #keep if marradult==1 & perweight!=0 

In [115]:
df2["hi_hsb"]=df2[["hi_hsb1"]].mean(axis=1) #by serial: egen hi_hsb = mean(hi_hsb1) \ keep if hi_hsb!=. & hi!=.
df3=df2.dropna(subset=["hi_hsb","hi"])

C:\Users\am513\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [116]:
#df3[df3["age"].str.contains("forward")]["age"]#.replace"85"

In [117]:
df4=df3.replace("85 (1968 and 1997 forward: 85+)",85) #age 85 was actually "85 (1968 and 1997 forward: 85+)" replaced with just 85

In [118]:
df4.astype({'age': 'int'})

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,...,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1,hi_hsb
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,...,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1.0
11,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,...,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
13,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7,...,0.0,1.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
15,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,8,...,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80603,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41166,...,1.0,5.0,85985.781250,87500.0,0.0,1.0,1.0,1.0,1.0,1.0
80608,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41168,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
80619,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,...,1.0,4.0,167844.531250,150000.0,1.0,1.0,1.0,1.0,1.0,1.0
80625,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,...,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0,1.0


In [121]:
df4.astype({"year":"int"})

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,...,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1,hi_hsb
4,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
7,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,...,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1.0
11,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,...,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
13,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7,...,0.0,1.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
15,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,8,...,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80603,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41166,...,1.0,5.0,85985.781250,87500.0,0.0,1.0,1.0,1.0,1.0,1.0
80608,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41168,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
80619,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,...,1.0,4.0,167844.531250,150000.0,1.0,1.0,1.0,1.0,1.0,1.0
80625,2009,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,...,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0,1.0


In [122]:
df4.astype({"age":"int"})

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,...,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1,hi_hsb
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,...,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1.0
11,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,6,...,0.0,3.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
13,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,7,...,0.0,1.0,41679.343750,42500.0,0.0,1.0,0.0,1.0,1.0,1.0
15,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,8,...,0.0,4.0,167844.531250,150000.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80603,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41166,...,1.0,5.0,85985.781250,87500.0,0.0,1.0,1.0,1.0,1.0,1.0
80608,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41168,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
80619,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41173,...,1.0,4.0,167844.531250,150000.0,1.0,1.0,1.0,1.0,1.0,1.0
80625,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41175,...,0.0,2.0,61102.972656,62500.0,0.0,1.0,0.0,0.0,1.0,1.0


In [137]:
df4["age"]=df4.age.astype(int)

In [140]:
df5=df4.query("age>=26 & age<=59 & marradult==1 & adltempl>=1") #gen angrist = ( age>=26 & age<=59 & marradult==1 & adltempl>=1 )

In [143]:
df5["famsize"]=df5.famsize.astype(str)

C:\Users\am513\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [161]:
df5

,year,inc1,inc2,inc3,inc4,inc5,inc6,inc7,inc8,serial,...,empl,hlth,inc,incmp,brooks,marradult,marradult_empl,adltempl,hi_hsb1,hi_hsb
4,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,3,...,1.0,4.0,19282.931641,17500.0,1.0,1.0,1.0,1.0,0.0,0.0
7,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,5,...,1.0,3.0,167844.531250,150000.0,1.0,1.0,1.0,2.0,1.0,1.0
21,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,10,...,1.0,4.0,85985.781250,87500.0,1.0,1.0,1.0,2.0,1.0,1.0
33,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,17,...,1.0,4.0,167844.531250,150000.0,1.0,1.0,1.0,1.0,1.0,1.0
36,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,19,...,1.0,4.0,61102.972656,62500.0,1.0,1.0,1.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80579,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41151,...,1.0,3.0,19282.931641,17500.0,1.0,1.0,1.0,2.0,1.0,1.0
80587,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41157,...,1.0,3.0,41679.343750,42500.0,1.0,1.0,1.0,2.0,1.0,1.0
80598,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41161,...,1.0,3.0,61102.972656,62500.0,1.0,1.0,1.0,2.0,1.0,1.0
80600,2009.0,19282.931641,41679.34375,61102.972656,85985.78125,167844.53125,25398.326172,109725.273438,70834.640625,41164,...,1.0,5.0,19282.931641,17500.0,1.0,1.0,1.0,2.0,0.0,0.0
